In [12]:
pip install pyspark

  Using cached pyspark-3.5.1.tar.gz (317.0 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488532 sha256=0c5de5b0205936b2564411acc9e768c74fd7955533d8b6af5a2f160bf5e0d855
  Stored in directory: c:\users\shaikpervez\appdata\local\pip\cache\wheels\da\78\6d\54350e0243f65f77dccf6ebe2ed5559faf6900559e904fb957
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [8]:
import sys
print(sys.version)

3.8.19 (default, Mar 20 2024, 19:55:45) [MSC v.1916 64 bit (AMD64)]


In [13]:
import mysql.connector
from pyspark.sql import SparkSession
# export PYSPARK_PYTHON=C:/Users/ShaikPervez/AppData/Local/Programs/Python/Python38/python.exe

In [14]:
spark = SparkSession.builder.appName("MySQL Connection").getOrCreate()

In [15]:
host="localhost"
port=3306
database="Customers"

In [16]:
mysql_url=f"jdbc:mysql://{host}:{port}/{database}"

In [17]:
mysql_properties = { "user" :"pervez", "password" :"password@123", "driver":"com.mysql.cj.jdbc.Driver"}

In [18]:
table_name="Customers_data"

In [19]:
print("JDBC URL : ",mysql_url)
print("MySQL Properties : ",mysql_properties)

JDBC URL :  jdbc:mysql://localhost:3306/Customers
MySQL Properties :  {'user': 'pervez', 'password': 'password@123', 'driver': 'com.mysql.cj.jdbc.Driver'}


In [20]:
try :
    df = spark.read.jdbc(url=mysql_url,table=table_name, properties=mysql_properties)
    df.show()
except Exception as e:
    print("Error",e)

+---+-------+----------+---+-----------------+
| id|   name|   address|age|Purchased_Product|
+---+-------+----------+---+-----------------+
|  1| Pervez|Tolichowki| 22|      Cricket bat|
|  2| Prasad|  Shaikpet| 26|     Cricket ball|
|  3|   Hari|   Suncity| 19|          Wickets|
|  4|Karthik|  Suryapet| 29|      Cricket bat|
|  5|Bhargav| Khajaguda| 24|     Cricket ball|
+---+-------+----------+---+-----------------+



In [21]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Purchased_Product: string (nullable = true)



In [22]:
df

DataFrame[id: int, name: string, address: string, age: int, Purchased_Product: string]

In [23]:
type(df)

pyspark.sql.dataframe.DataFrame

In [24]:
df.columns

['id', 'name', 'address', 'age', 'Purchased_Product']

In [25]:
df.head(3)

[Row(id=1, name='Pervez', address='Tolichowki', age=22, Purchased_Product='Cricket bat'),
 Row(id=2, name='Prasad', address='Shaikpet', age=26, Purchased_Product='Cricket ball'),
 Row(id=3, name='Hari', address='Suncity', age=19, Purchased_Product='Wickets')]

In [26]:
df.select(['name', 'id']).show()

+-------+---+
|   name| id|
+-------+---+
| Pervez|  1|
| Prasad|  2|
|   Hari|  3|
|Karthik|  4|
|Bhargav|  5|
+-------+---+



In [27]:
df.dtypes

[('id', 'int'),
 ('name', 'string'),
 ('address', 'string'),
 ('age', 'int'),
 ('Purchased_Product', 'string')]

In [28]:
df.describe().show()

+-------+------------------+-------+----------+------------------+-----------------+
|summary|                id|   name|   address|               age|Purchased_Product|
+-------+------------------+-------+----------+------------------+-----------------+
|  count|                 5|      5|         5|                 5|                5|
|   mean|               3.0|   null|      null|              24.0|             null|
| stddev|1.5811388300841898|   null|      null|3.8078865529319543|             null|
|    min|                 1|Bhargav| Khajaguda|                19|     Cricket ball|
|    max|                 5| Prasad|Tolichowki|                29|          Wickets|
+-------+------------------+-------+----------+------------------+-----------------+



In [29]:
df.columns

['id', 'name', 'address', 'age', 'Purchased_Product']

In [30]:
from pyspark.sql import functions as F

In [31]:
from pyspark import RDD

In [58]:
# Specify the column name from which you want to extract values
column_name = "Purchased_Product"

# Collect column values into a list
column_values = df.select(column_name).rdd.flatMap(lambda x: x).collect()


In [52]:
categ=[(i.split())[0] for i in column_values]

In [51]:
print(var)

['Cricket', 'Cricket', 'Wickets', 'Cricket', 'Cricket']


In [34]:
df.show()

+---+-------+----------+---+-----------------+
| id|   name|   address|age|Purchased_Product|
+---+-------+----------+---+-----------------+
|  1| Pervez|Tolichowki| 22|      Cricket bat|
|  2| Prasad|  Shaikpet| 26|     Cricket ball|
|  3|   Hari|   Suncity| 19|          Wickets|
|  4|Karthik|  Suryapet| 29|      Cricket bat|
|  5|Bhargav| Khajaguda| 24|     Cricket ball|
+---+-------+----------+---+-----------------+



In [164]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Purchased Product: string (nullable = false)



In [60]:
from pyspark.sql.functions import when

In [67]:
df = df.withColumn("Product_Category",
                   when(F.split(df[column_name], "\s+")[0]=="Wickets",F.lit("Cricket"))
                   .otherwise(F.lit("Cricket")))

In [68]:
df.show()

+---+-------+----------+---+-----------------+----------------+
| id|   name|   address|age|Purchased_Product|Product_Category|
+---+-------+----------+---+-----------------+----------------+
|  1| Pervez|Tolichowki| 22|      Cricket bat|         Cricket|
|  2| Prasad|  Shaikpet| 26|     Cricket ball|         Cricket|
|  3|   Hari|   Suncity| 19|          Wickets|         Cricket|
|  4|Karthik|  Suryapet| 29|      Cricket bat|         Cricket|
|  5|Bhargav| Khajaguda| 24|     Cricket ball|         Cricket|
+---+-------+----------+---+-----------------+----------------+



In [167]:
df.count()

3

In [219]:
from pyspark.sql.types import * 

In [169]:
df.groupby("Purchased Product").agg(F.countDistinct("id")).show()

+-----------------+---------+
|Purchased Product|count(id)|
+-----------------+---------+
|          Wickets|        1|
|      Cricket bat|        1|
|     Cricket ball|        1|
+-----------------+---------+



In [170]:
df = df.withColumnRenamed("Purchased Product","Purchased_Product")

In [171]:
df.show()

+---+---------------+-------------------+---+-----------------+
| id|           name|            address|age|Purchased_Product|
+---+---------------+-------------------+---+-----------------+
|  1|         Pervez|             Guntur| 22|     Cricket ball|
|  2|       John Doe|  123 Main St, City| 21|      Cricket bat|
|  3|Michael Johnson|789 Elm Rd, Village| 17|          Wickets|
+---+---------------+-------------------+---+-----------------+



In [70]:
from pyspark.sql import Row

In [79]:
new_row = Row(id=5, name='Chaithanya', address="Masab Tank", age=25, Purchased_Product="Tennis bat",Product_Category="Tennis")

In [80]:
df_new_row=spark.createDataFrame([new_row])

In [81]:
df_new_row.show()

+---+----------+----------+---+-----------------+----------------+
| id|      name|   address|age|Purchased_Product|Product_Category|
+---+----------+----------+---+-----------------+----------------+
|  5|Chaithanya|Masab Tank| 25|       Tennis bat|          Tennis|
+---+----------+----------+---+-----------------+----------------+



In [82]:
new_df=df.union(df_new_row)

In [253]:
new_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- age: long (nullable = true)
 |-- Purchased_Product: string (nullable = true)



In [85]:
new_df.write.jdbc(url=mysql_url, table="Customers_data", mode="overwrite",properties=mysql_properties)

Py4JJavaError: An error occurred while calling o375.jdbc.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 29.0 failed 1 times, most recent failure: Lost task 1.0 in stage 29.0 (TID 56) (172.16.1.200 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:192)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:166)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD

In [184]:
df.show()

+---+----+-------+---+-----------------+
| id|name|address|age|Purchased_Product|
+---+----+-------+---+-----------------+
+---+----+-------+---+-----------------+



In [234]:
import os
os.environ['PYSPARK_PYTHON'] = 'C:/Users/ShaikPervez/AppData/Local/Programs/Python/Python38'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'C:/Users/ShaikPervez/AppData/Local/Programs/Python/Python38'

In [22]:
import os

In [23]:
# Get PYSPARK_PYTHON value
pyspark_python = os.environ.get('PYSPARK_PYTHON')
print("PYSPARK_PYTHON:", pyspark_python)

# Get PYSPARK_DRIVER_PYTHON value
pyspark_driver_python = os.environ.get('PYSPARK_DRIVER_PYTHON')
print("PYSPARK_DRIVER_PYTHON:", pyspark_driver_python)


PYSPARK_PYTHON: None
PYSPARK_DRIVER_PYTHON: None
